In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('/content/news.csv')
print(df.shape)
df.head(5)

(6335, 4)


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
df['title'].unique()

array(['You Can Smell Hillary’s Fear',
       'Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO)',
       'Kerry to go to Paris in gesture of sympathy', ...,
       'Anti-Trump Protesters Are Tools of the Oligarchy     : Information',
       'In Ethiopia, Obama seeks progress on peace, security in East Africa',
       "Jeb Bush Is Suddenly Attacking Trump. Here's Why That Matters"],
      dtype=object)

In [ ]:
df['label'].unique()

array(['FAKE', 'REAL'], dtype=object)

In [ ]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [ ]:
data=df[['text','label']]

In [ ]:
data.head(5)


,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,It's primary day in New York and front-runners...,REAL


In [ ]:
data['text'].iloc[8]

'Hillary Clinton and Donald Trump made some inaccurate claims during an NBC “commander-in-chief” forum on military and veterans issues:\n\n• Clinton wrongly claimed Trump supported the war in Iraq after it started, while Trump was wrong, once again, in saying he was against the war before it started.\n\n•\xa0Trump said that President Obama set a “certain date” for withdrawing troops from Iraq, when that date was set before Obama was sworn in.\n\n•\xa0Trump said that Obama’s visits to China, Saudi Arabia and Cuba were “the first time in the history, the storied history of Air Force One” when “high officials” of a host country did not appear to greet the president. Not true.\n\n•\xa0Clinton said that Trump supports privatizing the Veterans Health Administration. That’s false. Trump said he supports allowing veterans to seek care at either public or private hospitals.\n\n•\xa0Trump said Clinton made “a terrible mistake on Libya” when she was secretary of State. But, at the time, Trump als

In [ ]:
import re
data['text']=data['text'].apply(lambda x:re.sub('[^a-zA-z0-9\s]','',x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data['text'].iloc[8]

'Hillary Clinton and Donald Trump made some inaccurate claims during an NBC commanderinchief forum on military and veterans issues\n\n Clinton wrongly claimed Trump supported the war in Iraq after it started while Trump was wrong once again in saying he was against the war before it started\n\n\xa0Trump said that President Obama set a certain date for withdrawing troops from Iraq when that date was set before Obama was sworn in\n\n\xa0Trump said that Obamas visits to China Saudi Arabia and Cuba were the first time in the history the storied history of Air Force One when high officials of a host country did not appear to greet the president Not true\n\n\xa0Clinton said that Trump supports privatizing the Veterans Health Administration Thats false Trump said he supports allowing veterans to seek care at either public or private hospitals\n\n\xa0Trump said Clinton made a terrible mistake on Libya when she was secretary of State But at the time Trump also supported US action that led to th

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words=set(stopwords.words('english'))

In [ ]:
data['text']=data['text'].apply(lambda x:' '.join([word for word in x.split() if word not in stop_words]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data['text'].iloc[8]

'Hillary Clinton Donald Trump made inaccurate claims NBC commanderinchief forum military veterans issues Clinton wrongly claimed Trump supported war Iraq started Trump wrong saying war started Trump said President Obama set certain date withdrawing troops Iraq date set Obama sworn Trump said Obamas visits China Saudi Arabia Cuba first time history storied history Air Force One high officials host country appear greet president Not true Clinton said Trump supports privatizing Veterans Health Administration Thats false Trump said supports allowing veterans seek care either public private hospitals Trump said Clinton made terrible mistake Libya secretary State But time Trump also supported US action led removal Moammar Gadhafi power Trump cherrypicked Clintons words claimed Clinton said vets treated essentially fine Clinton said problems Department Veterans Affairs widespread Republicans claimed went acknowledge problems including issue wait times doctors The forum sponsored NBC News Iraq

In [ ]:
max_features=5000
tokenizer=Tokenizer(num_words=max_features,split=' ')
tokenizer.fit_on_texts(data['text'].values)

In [ ]:
seq=tokenizer.texts_to_sequences(data['text'].values)

In [ ]:
seq[8]

[19,
 5,
 55,
 3,
 77,
 700,
 1903,
 2431,
 116,
 1031,
 231,
 5,
 843,
 3,
 1187,
 64,
 289,
 636,
 3,
 615,
 169,
 64,
 636,
 3,
 2,
 15,
 22,
 333,
 790,
 1878,
 911,
 289,
 1878,
 333,
 22,
 4578,
 3,
 2,
 264,
 4338,
 499,
 724,
 1275,
 1638,
 30,
 24,
 236,
 236,
 493,
 317,
 9,
 322,
 160,
 1170,
 62,
 969,
 15,
 543,
 415,
 5,
 2,
 3,
 2315,
 1031,
 202,
 206,
 95,
 1093,
 3,
 2,
 2315,
 1773,
 1031,
 1481,
 314,
 419,
 89,
 327,
 4272,
 3,
 2,
 5,
 77,
 2432,
 1849,
 1148,
 178,
 13,
 11,
 24,
 3,
 14,
 1187,
 7,
 424,
 548,
 4339,
 124,
 3,
 84,
 478,
 843,
 5,
 2,
 2304,
 1653,
 2132,
 5,
 2,
 598,
 144,
 1031,
 1312,
 2789,
 68,
 843,
 405,
 3096,
 598,
 115,
 207,
 1470,
 137,
 1995,
 1,
 2431,
 4511,
 1903,
 48,
 289,
 1276,
 1031,
 98,
 520,
 2738,
 1008,
 1566,
 493,
 1231,
 10,
 163,
 249,
 208,
 221,
 1170,
 2252,
 252,
 116,
 1031,
 1310,
 2497,
 119,
 3,
 2,
 1840,
 64,
 289,
 5,
 843,
 1187,
 289,
 64,
 636,
 1,
 1320,
 45,
 71,
 419,
 119,
 457,
 797,
 972,
 70,
 

In [ ]:
print('length =',len(seq[8]))


length = 1251


In [ ]:
seq=pad_sequences(seq)

In [ ]:
print(seq[8])
print('length ',len(seq[8]))

[  0   0   0 ...   9 863 900]
length  9615


In [ ]:
seq.shape

(6335, 9615)

In [ ]:
y=pd.get_dummies(data['label'],drop_first=True).values

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(seq,y,test_size=0.2)

In [ ]:
print('x_train:',x_train.shape,'x_test:',x_test.shape)
print('y_train:',y_train.shape,'y_test:',y_test.shape)

x_train: (5068, 9615) x_test: (1267, 9615)
y_train: (5068, 1) y_test: (1267, 1)


In [ ]:
embed_dimentions=128

In [ ]:
model=Sequential()
model.add(Embedding(max_features,embed_dimentions,input_length=seq.shape[1]))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 9615, 128)         640000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 771,713
Trainable params: 771,713
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#run_opts = tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True)

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=64,epochs=5)

Epoch 1/5
80/80 [==============================] - 3309s 41s/step - loss: 0.5977 - accuracy: 0.6797 - val_loss: 0.2855 - val_accuracy: 0.8895
Epoch 2/5
80/80 [==============================] - 3171s 40s/step - loss: 0.4866 - accuracy: 0.8338 - val_loss: 0.4358 - val_accuracy: 0.8287
Epoch 3/5
80/80 [==============================] - 3254s 41s/step - loss: 0.2542 - accuracy: 0.9264 - val_loss: 0.2826 - val_accuracy: 0.8856
Epoch 4/5
80/80 [==============================] - 3299s 41s/step - loss: 0.1384 - accuracy: 0.9539 - val_loss: 0.2764 - val_accuracy: 0.8966
Epoch 5/5
80/80 [==============================] - 3269s 41s/step - loss: 0.1301 - accuracy: 0.9556 - val_loss: 0.3579 - val_accuracy: 0.8895


In [ ]:
model.save('news.h5')